In [1]:
import os
from PIL import Image

In [2]:
import numpy as np

In [3]:
from keras.models import Sequential,Model

from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Lambda, Flatten, Dense
from keras.layers.merge import Concatenate

from keras.initializers import glorot_uniform
from keras.regularizers import l2

from keras.optimizers import Adam
from keras import losses
from keras import metrics

from keras import backend as K

from keras.utils import plot_model

from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [4]:
def initialize_weights(shape, name=None):
    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)

In [5]:
def initialize_bias(shape, name=None):
    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)

In [6]:
def get_siamese_model(input_shape):
    model = Sequential()
    
    model.add(Conv2D(64, (10,10), activation='relu', input_shape=input_shape,
                     kernel_initializer=initialize_weights, 
                     kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    
    model.add(Conv2D(128, (7,7), activation='relu',
                     kernel_initializer=initialize_weights, bias_initializer=initialize_bias,
                     kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    
    model.add(Conv2D(128, (4,4), activation='relu', 
                     kernel_initializer=initialize_weights, bias_initializer=initialize_bias, 
                     kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    
    model.add(Conv2D(256, (4,4), activation='relu', 
                     kernel_initializer=initialize_weights, bias_initializer=initialize_bias, 
                     kernel_regularizer=l2(2e-4)))
    model.add(Flatten())
    
    model.add(Dense(4096, activation='sigmoid',
                   kernel_initializer=initialize_weights, bias_initializer=initialize_bias,
                   kernel_regularizer=l2(1e-3)))
    
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    encoded_l = model(left_input)
    encoded_r = model(right_input)
        
    euclidian_distance = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))([encoded_l, encoded_r])
    prediction = Dense(1,activation='sigmoid', bias_initializer=initialize_bias)(euclidian_distance)
    
    siamese_model = Model(inputs=[left_input,right_input], outputs=prediction)
    
    siamese_model.compile(loss="binary_crossentropy",optimizer=Adam(lr = 0.00006))
    
    return siamese_model

In [7]:
model = get_siamese_model((224, 224, 3))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 4096)         420642112   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None

In [8]:
plot_model(model, show_shapes=True)

In [9]:
import pickle
pickle_in = open("../Data/image_data.pkl","rb")
df_from_pickle = pickle.load(pickle_in)
pickle_in.close()

In [10]:
df_from_pickle.shape

(12122, 4)

In [11]:
msk_train_test = np.random.rand(len(df_from_pickle)) < 0.7
training_data = df_from_pickle[msk_train_test]
testing_data = df_from_pickle[~msk_train_test]

In [12]:
msk_train_val = np.random.rand(len(training_data)) < 0.8
train_df = training_data[msk_train_val]
validation_df = training_data[~msk_train_val]
test_df = testing_data

In [13]:
train_df
#validation_df
#test_df

,combination_pair_from_dir,left_image_path,right_image_path,is_related
2,F0002/MID1-F0002/MID3,../Data/recognizing-faces-in-the-wild/train/F0...,../Data/recognizing-faces-in-the-wild/train/F0...,1
7,F0005/MID1-F0005/MID2,../Data/recognizing-faces-in-the-wild/train/F0...,../Data/recognizing-faces-in-the-wild/train/F0...,1
12,F0009/MID1-F0009/MID2,../Data/recognizing-faces-in-the-wild/train/F0...,../Data/recognizing-faces-in-the-wild/train/F0...,1
13,F0009/MID1-F0009/MID2,../Data/recognizing-faces-in-the-wild/train/F0...,../Data/recognizing-faces-in-the-wild/train/F0...,1
14,F0009/MID1-F0009/MID3,../Data/recognizing-faces-in-the-wild/train/F0...,../Data/recognizing-faces-in-the-wild/train/F0...,1
17,F0009/MID1-F0009/MID4,../Data/recognizing-faces-in-the-wild/train/F0...,../Data/recognizing-faces-in-the-wild/train/F0...,1
20,F0009/MID1-F0009/MID6,../Data/recognizing-faces-in-the-wild/train/F0...,../Data/recognizing-faces-in-the-wild/train/F0...,1
21,F0009/MID1-F0009/MID6,../Data/recognizing-faces-in-the-wild/train/F0...,../Data/recognizing-faces-in-the-wild/train/F0...,1
22,F0009/MID1-F0009/MID7,../Data/recognizing-faces-in-the-wild/train/F0...,../Data/recognizing-faces-in-the-wild/train/F0...,0
23,F0009/MID1-F0009/MID7,../Data/recognizing-faces-in-the-wild/train/F0...,../Data/recognizing-faces-in-the-wild/train/F0...,0


In [14]:
print(train_df.shape)
print(validation_df.shape)
print(test_df.shape)

(6827, 4)
(1715, 4)
(3580, 4)


In [15]:
train_df.shape[0]

6827

In [16]:
train_df.shape[0]+validation_df.shape[0]+test_df.shape[0]

12122

In [17]:
def batches_generator(batch_size, data_frame, shuffle=True):
    """Generates padded batches of tokens and tags."""
    
    n_samples = data_frame.shape[0]
    if shuffle:
        order = np.random.permutation(n_samples)
    else:
        order = np.arange(n_samples)

    n_batches = n_samples // batch_size
    
    for k in range(n_batches):
        batch_start = k * batch_size
        batch_end = min((k + 1) * batch_size, n_samples)
        current_batch_size = batch_end - batch_start
      
        pairs = [np.zeros((batch_size, 224, 224, 3)) for i in range(2)]
        targets = np.zeros((batch_size,))
        
        pair_index=0
        for idx in order[batch_start: batch_end]:
            left_image_array=np.asarray(Image.open(data_frame.iloc[idx]['left_image_path']))
            pairs[0][pair_index,:,:,:] = left_image_array
                        
            right_image_array=np.asarray(Image.open(data_frame.iloc[idx]['right_image_path']))
            pairs[1][pair_index,:,:,:] = right_image_array                        
            
            targets[pair_index] = data_frame.iloc[idx]['is_related']

            pair_index = pair_index+1
            
        yield pairs, targets


In [18]:
batch = batches_generator(10, train_df, shuffle=False)

In [19]:
next_item = next(batch)

In [20]:
len(next_item)

2

In [21]:
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto', restore_best_weights=True)
checkpoint = ModelCheckpoint(filepath='model_{epoch:02d}_{val_loss:.2f}.hdf5')

In [22]:
history = model.fit_generator(
                    generator= batches_generator(10, train_df, shuffle=True), 
                    epochs=500,
                    steps_per_epoch = 1,
                    verbose=2, 
                    callbacks=[monitor], 
                    validation_data = batches_generator(10, validation_df, shuffle=True),
                    validation_steps = 1 )

Instructions for updating:
Use tf.cast instead.
Epoch 1/500
 - 103s - loss: 42.6974 - val_loss: 42.5106
Epoch 2/500
 - 8s - loss: 42.5325 - val_loss: 42.5572
Epoch 3/500
 - 8s - loss: 42.3016 - val_loss: 42.5063
Epoch 4/500
 - 8s - loss: 42.3357 - val_loss: 42.0096
Epoch 5/500
 - 8s - loss: 42.0963 - val_loss: 41.8677
Epoch 6/500
 - 8s - loss: 41.8450 - val_loss: 41.6794
Epoch 7/500
 - 9s - loss: 41.5733 - val_loss: 41.4380
Epoch 8/500
 - 8s - loss: 41.2914 - val_loss: 41.3642
Epoch 9/500
 - 8s - loss: 41.3633 - val_loss: 41.1498
Epoch 10/500
 - 8s - loss: 40.9520 - val_loss: 41.1835
Epoch 11/500
 - 8s - loss: 40.8372 - val_loss: 40.7689
Epoch 12/500
 - 8s - loss: 40.7782 - val_loss: 40.4494
Epoch 13/500
 - 8s - loss: 40.4247 - val_loss: 40.2070
Epoch 14/500
 - 8s - loss: 40.2785 - val_loss: 39.9963
Epoch 15/500
 - 8s - loss: 40.0536 - val_loss: 39.8253
Epoch 16/500
 - 8s - loss: 39.7700 - val_loss: 39.6110
Epoch 17/500
 - 8s - loss: 39.5843 - val_loss: 39.3864
Epoch 18/500
 - 8s - los

 - 8s - loss: 19.0842 - val_loss: 19.0583
Epoch 145/500
 - 8s - loss: 19.0482 - val_loss: 18.9684
Epoch 146/500
 - 8s - loss: 19.0563 - val_loss: 18.8684
Epoch 147/500
 - 8s - loss: 18.7847 - val_loss: 18.6045
Epoch 148/500
 - 8s - loss: 18.6672 - val_loss: 18.5332
Epoch 149/500
 - 8s - loss: 18.6945 - val_loss: 18.5635
Epoch 150/500
 - 8s - loss: 18.5287 - val_loss: 18.4419
Epoch 151/500
 - 8s - loss: 18.4719 - val_loss: 18.3884
Epoch 152/500
 - 8s - loss: 18.4404 - val_loss: 18.2508
Epoch 153/500
 - 8s - loss: 18.2060 - val_loss: 18.1895
Epoch 154/500
 - 8s - loss: 18.2725 - val_loss: 18.0021
Epoch 155/500
 - 8s - loss: 18.1052 - val_loss: 17.9049
Epoch 156/500
 - 8s - loss: 17.8307 - val_loss: 17.8128
Epoch 157/500
 - 8s - loss: 17.8264 - val_loss: 17.8276
Epoch 158/500
 - 8s - loss: 17.6949 - val_loss: 17.6636
Epoch 159/500
 - 8s - loss: 17.6621 - val_loss: 17.6029
Epoch 160/500
 - 8s - loss: 17.5567 - val_loss: 17.4277
Epoch 161/500
 - 8s - loss: 17.4810 - val_loss: 17.2112
Epoch 

StopIteration: 

In [23]:
model.save_weights("finalmodels/final.h5")

OSError: Unable to create file (unable to open file: name = 'finalmodels/final.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 302)